In [148]:
import torch
import torch.nn.functional as F

import matplotlib.pyplot as plt
%matplotlib inline

In [149]:
# read all the words
words = open('names.txt', 'r').read().splitlines()

In [150]:
len(words)

32033

In [151]:
# build the vocabulary of chars and mapping to and from integers
chars = sorted(list(set(''.join(words))))

stoi = {s:i+1 for i, s in enumerate(chars)}
stoi['.'] = 0

itos = {i:s for s,i in stoi.items()}

In [208]:
# buid the data set

def build_dataset(words):
    BLOCK_SIZE = 3 # context length: how many char we take to predict next char

    X, Y = [], []

    for w in words:
        context = [0] * BLOCK_SIZE
        for ch in list(w) + ['.']:
            ix = stoi[ch]
            
            X.append(context)
            Y.append(ix)
            
            # update the context with next char
            context = context[1:] + [ix]

    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(X.shape, Y.shape)
    return X, Y

In [209]:
# create train, validate and test split
import random
random.seed(42)

# shuffle the words
random.shuffle(words)

# 80%
n1 = int(0.8*len(words))
# 90%
n2 = int(0.9*len(words))

# train, dev and test data set
Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1: n2])
Xte, Yte = build_dataset(words[n2:])

torch.Size([182437, 3]) torch.Size([182437])
torch.Size([22781, 3]) torch.Size([22781])
torch.Size([22928, 3]) torch.Size([22928])


In [210]:
Xtr.shape, Xtr.dtype, Ytr.shape, Ytr.dtype

(torch.Size([182437, 3]), torch.int64, torch.Size([182437]), torch.int64)

In [211]:
# generator
g = torch.Generator().manual_seed(2147483647)

In [212]:
# create embeddings
C = torch.randn((27, 10), generator=g)

# hidden layers
W1 = torch.randn((30, 200), generator=g)
b1 = torch.randn(200, generator=g)

W2 = torch.randn((200, 27), generator=g)
b2 = torch.randn(27, generator=g)

parameters = [C, W1, b1, W2, b2]

In [213]:
# number of parameters

sum(p.nelement() for p in parameters)

11897

In [214]:
for p in parameters:
    p.requires_grad = True

In [215]:
lre = torch.linspace(-3, 0, 1000)
lrs = 10**lre
lri = []
lossi = []
stepi = []

In [ ]:
for i in range(200000):
    
    # minibatch construct
    ix = torch.randint(0, Xtr.shape[0], (32, ))
    
    # forward pass

    # embedding
    emb = C[Xtr[ix]]

    # tanh
    h = torch.tanh(emb.view(-1, 30) @ W1 + b1)

    # logits
    logits = h @ W2 + b2

    # # soft max
    loss = F.cross_entropy(logits, Ytr[ix])
    
    # backward pass
    for p in parameters:
        p.grad = None

    loss.backward()

    # update 
    lr = 0.1 if i < 100000 else 0.01
    # optimim
    for p in parameters:
        p.data += -lr * p.grad
        
    # track stats
    stepi.append(i)
    lossi.append(loss.log10().item())

In [ ]:
plt.plot(stepi, lossi)

In [ ]:
emb = C[Xtr]
h = torch.tanh(emb.view(-1, 30) @ W1 + b1)
logits = h @ W2 + b2
loss = F.cross_entropy(logits, Ytr)
loss

In [ ]:
emb = C[Xdev]
h = torch.tanh(emb.view(-1, 30) @ W1 + b1)
logits = h @ W2 + b2
loss = F.cross_entropy(logits, Ydev)
loss